# So sánh Apriori vs FP-Growth

**Mục đích:** So sánh hiệu năng và chất lượng luật (số luật, lift, confidence, support) giữa hai thuật toán Apriori và FP-Growth trên dữ liệu `basket_bool`.

**Tổng quan:** Notebook chạy hai thuật toán với nhiều giá trị `min_support`, thu thập thời gian chạy, số lượng itemset, số luật, và độ dài trung bình của itemset; sau đó lưu bảng so sánh và hiển thị các biểu đồ để phân tích.

**Hướng dẫn nhanh:** Thay đổi các tham số trong cell `PARAMETERS` ở đầu notebook rồi chạy tuần tự các cell (hoặc chạy toàn bộ) để thu thập kết quả mới.

In [ ]:
# PARAMETERS (for papermill)

BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"

# Dải min_support để so sánh độ nhạy tham số (Q2)
MIN_SUPPORT_LIST = [0.02, 0.015, 0.01, 0.0075, 0.005]

# Tham số chung để sinh luật và lọc
MIN_CONFIDENCE = 0.3
MIN_LIFT = 1.2

# Giới hạn độ dài itemset khi mining (đồng nhất)
MAX_LEN = 3

# Output bảng so sánh
COMPARE_OUTPUT_PATH = "data/processed/compare_apriori_fpgrowth.csv"

# Plot
PLOT_RUNTIME_SENSITIVITY = True
PLOT_RULES_SCATTER = True

# Plotly interactive
PLOT_PLOTLY_SCATTER = True

## Tham số (PARAMETERS)

**Giải thích các tham số chính:**

- `BASKET_BOOL_PATH`: đường dẫn tới file Parquet chứa ma trận boolean (mỗi hàng là 1 giỏ, mỗi cột là 1 sản phẩm).
- `MIN_SUPPORT_LIST`: danh sách các giá trị min_support để thử nghiệm (sensitivity analysis).
- `MIN_CONFIDENCE`, `MIN_LIFT`: ngưỡng lọc luật (sau khi sinh).
- `MAX_LEN`: độ dài tối đa của itemset (số sản phẩm trong 1 tập mục).
- `COMPARE_OUTPUT_PATH`: nơi lưu kết quả so sánh (CSV).
- `PLOT_*` flags: bật/tắt các biểu đồ (matplotlib / plotly).

> Thay đổi các giá trị này để kiểm tra ảnh hưởng của tham số tới hiệu năng và số luật sinh ra.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

from mlxtend.frequent_patterns import apriori, association_rules
from apriori_library import FPGrowthMiner

## Thiết lập môi trường & Imports

Cell này thực hiện các bước sau:

- Nạp các thư viện cần thiết (pandas, matplotlib, seaborn, plotly, mlxtend, v.v.).
- Thiết lập `project_root` và thêm `src` vào `sys.path` để import `apriori_library.FPGrowthMiner`.

> Lưu ý: nếu bạn chạy notebook từ thư mục `notebooks`, bước xác định `project_root` đảm bảo import từ `src` hoạt động đúng.

In [ ]:
basket_bool = pd.read_parquet(BASKET_BOOL_PATH)
print("basket_bool shape:", basket_bool.shape)

## Hàm helper (mô tả)

Các hàm chính ở đây dùng để chạy thử nghiệm cho mỗi thuật toán và trả về các chỉ số đánh giá:

- `avg_itemset_len(freq_df)` — tính độ dài trung bình của các itemset trong DataFrame `freq`.
- `run_apriori_once(basket_bool, min_support, max_len, min_conf, min_lift)` — chạy Apriori, sinh itemsets và luật (sử dụng `association_rules` từ mlxtend), lọc theo lift; trả về dict với các trường: `time_seconds`, `n_itemsets`, `n_rules`, `avg_itemset_len`, `rules`.
- `run_fpgrowth_once(...)` — tương tự nhưng sử dụng `FPGrowthMiner` từ `src/apriori_library.py`. Nếu miner không cung cấp `association_rules`, function sẽ fallback sang `generate_rules`.

Những dict trả về được dùng để xây dựng bảng so sánh `compare_df`.

## Nạp dữ liệu (basket_bool)

Ở đây chúng ta đọc `basket_bool` từ `BASKET_BOOL_PATH` (Parquet). Dữ liệu có dạng ma trận boolean: mỗi hàng = một giỏ hàng, mỗi cột = một sản phẩm (True/False).

- Hãy kiểm tra `basket_bool.shape` để biết kích thước dữ liệu.
- Nếu bạn chưa có file này, chạy notebook `basket_preparation.ipynb` để tạo file `basket_bool.parquet`.

In [ ]:
def avg_itemset_len(freq_df: pd.DataFrame) -> float:
    if freq_df is None or freq_df.empty:
        return 0.0
    return float(freq_df["itemsets"].apply(len).mean())

def run_apriori_once(basket_bool: pd.DataFrame, min_support: float, max_len: int, min_conf: float, min_lift: float):
    t0 = time.perf_counter()

    freq = apriori(basket_bool, min_support=min_support, use_colnames=True, max_len=max_len)
    rules = association_rules(freq, metric="confidence", min_threshold=min_conf)
    rules = rules[rules["lift"] >= min_lift].copy()

    t1 = time.perf_counter()

    return {
        "time_seconds": t1 - t0,
        "n_itemsets": int(len(freq)),
        "n_rules": int(len(rules)),
        "avg_itemset_len": avg_itemset_len(freq),
        "rules": rules,
    }

def run_fpgrowth_once(basket_bool: pd.DataFrame, min_support: float, max_len: int, min_conf: float, min_lift: float):
    miner = FPGrowthMiner(basket_bool=basket_bool)

    t0 = time.perf_counter()

    freq = miner.run(min_support=min_support, use_colnames=True, max_len=max_len)

    if hasattr(miner, "association_rules"):
        rules = miner.association_rules(freq, metric="confidence", min_threshold=min_conf)
    else:
        # fallback nếu bạn dùng generate_rules
        rules = miner.generate_rules(freq, min_confidence=min_conf, metric="confidence")

    rules = rules[rules["lift"] >= min_lift].copy()

    t1 = time.perf_counter()

    return {
        "time_seconds": t1 - t0,
        "n_itemsets": int(len(freq)),
        "n_rules": int(len(rules)),
        "avg_itemset_len": avg_itemset_len(freq),
        "rules": rules,
    }

## Thực nghiệm & Lưu kết quả

Quy trình thực nghiệm:

1. Duyệt qua từng `min_support` trong `MIN_SUPPORT_LIST`.
2. Với mỗi giá trị, chạy `run_apriori_once` và `run_fpgrowth_once` để tính thời gian, số itemset, số luật, và avg itemset length.
3. Lưu kết quả vào danh sách `rows` và tạo `compare_df` (DataFrame tổng hợp).
4. Lưu bảng `compare_df` vào `COMPARE_OUTPUT_PATH` (CSV) để phân tích/so sánh sau này.

> Lưu ý: snapshot của luật tại `min_support=0.01` được dùng để vẽ biểu đồ phân bố luật (scatter).

In [ ]:
rows = []
snapshots = {}  # rules snapshot tại min_support=0.01 để vẽ scatter

for ms in MIN_SUPPORT_LIST:
    res_a = run_apriori_once(basket_bool, ms, MAX_LEN, MIN_CONFIDENCE, MIN_LIFT)
    res_f = run_fpgrowth_once(basket_bool, ms, MAX_LEN, MIN_CONFIDENCE, MIN_LIFT)

    rows.append({
        "algo": "Apriori",
        "min_support": ms,
        "time_seconds": res_a["time_seconds"],
        "n_itemsets": res_a["n_itemsets"],
        "n_rules": res_a["n_rules"],
        "avg_itemset_len": res_a["avg_itemset_len"],
    })

    rows.append({
        "algo": "FP-Growth",
        "min_support": ms,
        "time_seconds": res_f["time_seconds"],
        "n_itemsets": res_f["n_itemsets"],
        "n_rules": res_f["n_rules"],
        "avg_itemset_len": res_f["avg_itemset_len"],
    })

    if abs(ms - 0.01) < 1e-12:
        snapshots["Apriori"] = res_a["rules"]
        snapshots["FP-Growth"] = res_f["rules"]

compare_df = pd.DataFrame(rows).sort_values(["min_support", "algo"], ascending=[False, True]).reset_index(drop=True)
compare_df

In [ ]:
os.makedirs(os.path.dirname(COMPARE_OUTPUT_PATH), exist_ok=True)
compare_df.to_csv(COMPARE_OUTPUT_PATH, index=False)
print("Saved compare table to:", COMPARE_OUTPUT_PATH)

## Kết quả & Biểu đồ

- **Runtime sensitivity**: biểu đồ thời gian chạy (time_seconds) theo `min_support` để so sánh hiệu năng giữa Apriori và FP-Growth.
- **Rules scatter**: biểu đồ scatter (support vs confidence) cho snapshot luật tại `min_support=0.01` để so sánh phân bố luật giữa hai thuật toán.
- **Plotly interactive**: nếu `PLOT_PLOTLY_SCATTER=True`, sẽ hiện biểu đồ tương tác (kích thước điểm theo `lift` và hover hiển thị antecedents/consequents/lift).

> Kiểm tra file `COMPARE_OUTPUT_PATH` (CSV) để xem bảng số liệu tóm tắt thu được từ các thử nghiệm.

In [ ]:
if PLOT_RUNTIME_SENSITIVITY:
    pivot_time = compare_df.pivot(index="min_support", columns="algo", values="time_seconds").sort_index(ascending=False)

    ax = pivot_time.plot(marker="o")
    ax.set_xlabel("min_support")
    ax.set_ylabel("time_seconds")
    ax.set_title("Runtime sensitivity: Apriori vs FP-Growth")
    plt.show()

In [ ]:
if PLOT_RULES_SCATTER:
    ar = snapshots.get("Apriori")
    fp = snapshots.get("FP-Growth")

    plt.figure()
    if ar is not None and not ar.empty:
        plt.scatter(ar["support"], ar["confidence"], alpha=0.5, label="Apriori")
    if fp is not None and not fp.empty:
        plt.scatter(fp["support"], fp["confidence"], alpha=0.5, label="FP-Growth")

    plt.xlabel("support")
    plt.ylabel("confidence")
    plt.title("Rules distribution (min_support=0.01)")
    plt.legend()
    plt.show()

In [ ]:
if PLOT_PLOTLY_SCATTER:
    ar = snapshots.get("Apriori")
    fp = snapshots.get("FP-Growth")

    frames = []
    if ar is not None and not ar.empty:
        tmp = ar.copy()
        tmp["algo"] = "Apriori"
        frames.append(tmp)
    if fp is not None and not fp.empty:
        tmp = fp.copy()
        tmp["algo"] = "FP-Growth"
        frames.append(tmp)

    if frames:
        df_plot = pd.concat(frames, ignore_index=True)
        fig = px.scatter(
            df_plot,
            x="support",
            y="confidence",
            color="algo",
            size="lift",
            hover_data=["antecedents", "consequents", "lift"],
            title="Rules (Interactive): Apriori vs FP-Growth (min_support=0.01)"
        )
        fig.show()